<a href="https://colab.research.google.com/github/acorrales650/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-14 21:39:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-03-14 21:39:30 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *

In [6]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_df_20 = df.filter('total_votes > 19')
votes_df_20.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [7]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percentage_vote_df = votes_df_20.withColumn('percentage_vote', votes_df_20['helpful_votes']/ votes_df_20['total_votes'])
top_50_df = percentage_vote_df.filter('percentage_vote >= 0.50')
top_50_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   percentage_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|0.9120879120879121|
|         US|    1243530|R13C5INE1RTZP6|B00G

In [8]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_yes_df = top_50_df.filter('vine == "Y"')
vine_yes_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   percentage_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   21155796|R3N0XV9267NOXV|B00ZKTVI66|     155258416|Graco LiteRider C...|            Baby|          5|           41|         47|   Y|                N|No more disturbin...|This version of t...| 2015-08-31|0.8723404255319149|
|         US|   25087234| RSA6JQ346JZHZ|B00S

In [9]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
vine_no_df = top_50_df.filter('vine == "N"')
vine_no_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   percentage_vote|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|0.9120879120879121|
|         US|    1243530|R13C5INE1RTZP6|B00G

In [10]:
# Number of 5-star reviews
five_stars_df = top_50_df.filter('star_rating == "5"')
five_stars_df.count()

12235

In [11]:
# 5-star reviews paid and unpaid
vine_y_five = vine_yes_df.filter('star_rating == "5"')
vine_n_five = vine_no_df.filter('star_rating == "5"')

In [12]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_votes = votes_df_20.count()
total_5_star = five_stars_df.count()
vine_y_total = vine_yes_df.count()
vine_n_total = vine_no_df.count()
vine_y5_total = vine_y_five.count()
vine_n5_total = vine_n_five.count()

In [13]:
# Percentage of 5 star reviews paid
vine_five_paid = (vine_y5_total / vine_y_total) * 100
vine_five_paid

43.628509719222464

In [14]:
# Percentage of 5 star reviews unpaid
vine_five_unpaid = (vine_n5_total / vine_n_total) * 100
vine_five_unpaid

47.95170160197657

In [15]:
import pandas as pd


In [18]:
summary_visual = {'Queries' :['Total Number of Reviews', ' Number of 5 Star Votes', 'Percent of 5 Star Votes - Paid', 'Percent of 5 Star Votes - Unpaid'], 
           'Results': [total_votes, total_5_star, vine_five_paid, vine_five_unpaid]}
summary_stats_df = pd.DataFrame(summary_visual)
summary_stats_df

,Queries,Results
0,Total Number of Reviews,27389.000000
1,Number of 5 Star Votes,12235.000000
2,Percent of 5 Star Votes - Paid,43.628510
3,Percent of 5 Star Votes - Unpaid,47.951702
